In [3]:
!pip install torch
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00


In [4]:
from transformers import BertTokenizer, BertModel
import torch
from datasets import load_dataset
from tqdm.notebook import tqdm

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
def get_datasets():
  imdb = load_dataset('imdb')
  sst2 = load_dataset('sst2')
  yelp_review_full = load_dataset('yelp_review_full')
  return {'imdb': imdb}#, 'sst2': sst2, 'yelp': yelp_review_full}

datasets_dict = get_datasets()

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
model = model.to(device)

In [56]:
split = 0.1

for ds_name, dataset in datasets_dict.items():

  print(f'------------ {ds_name} ------------')


  len_train = len(dataset['train'])
  dim_split = int(len_train * split)

  range_splits = [(idx, len_train) if idx + dim_split > len_train else (idx, idx + dim_split) for idx in range(0, len_train, dim_split)]

  for idx, (strat_range, end_range) in enumerate(range_splits):

    embeddings_tensor = torch.empty((0,768)).to(device)
    outputs = torch.empty((0,768)).to(device)

    torch.save(embeddings_tensor, f'{ds_name}/embeddings_{idx}.pt')


    print(f'---------- Split {idx} ----------')

    ds_dict = dict(dataset['train'][strat_range:end_range].items())

    if 'text' in ds_dict: ds_dict = ds_dict['text']
    elif 'sentence' in ds_dict: ds_dict = value_texts['sentence']
    else: raise Exception('Invalid key for datasets')


    for text in tqdm(ds_dict, total = len(ds_dict), leave=False):

      embeddings_tensor = torch.load(f'{ds_name}/embeddings_{idx}.pt')

      encoded_text = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)

      outputs = model(**encoded_text, output_hidden_states=True).last_hidden_state[:, 0, :]

      embeddings_tensor = torch.cat((embeddings_tensor, outputs), dim=0)

      torch.save(embeddings_tensor, f'{ds_name}/embeddings_{idx}.pt')


------------ imdb ------------
---------- Split 0 ----------


  0%|          | 0/2500 [00:00<?, ?it/s]

---------- Split 1 ----------


  0%|          | 0/2500 [00:00<?, ?it/s]

---------- Split 2 ----------


...

  0%|          | 0/2500 [00:00<?, ?it/s]

---------- Split 9 ----------


  0%|          | 0/2500 [00:00<?, ?it/s]

------------ sst2 ------------
---------- Split 0 ----------


NameError: ignored

In [ ]:
for ds_name in list(datasets_dict.keys())
  to_array = np.empty((0,768))
  for idx in range(10):
    to_array = np.vstack((to_array,
                          torch.load(f'{ds_name}/embeddings_{idx}.pt').numpy()))
  np.save(f'{ds_name}/embeddings.npy', to_array)

In [ ]:
torch.load('/content/imdb_embeddings.pt').size()